In [1]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import seaborn as sns
import json
import fileinput
import numpy as np
from pprint import pprint
from wordcloud import WordCloud #For issues during installing wordcloud, try: https://stackoverflow.com/questions/41409570/cant-install-wordcloud-in-python-anaconda
from textblob_de import TextBlobDE
import re
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
from datetime import date
import re
import string
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

2022-06-04 13:48:45.163 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")

olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)
print("Datensatzlänge: " + str(olafScholzTwitter.info()))

print(olafScholzTwitter.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   http_status   65 non-null     int64 
 1   account_name  65 non-null     object
 2   account_data  65 non-null     object
 3   params        65 non-null     object
 4   response      65 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
Datensatzlänge: None
   http_status account_name  \
0          200   OlafScholz   
1          200   OlafScholz   
2          200   OlafScholz   
3          200   OlafScholz   
4          200   OlafScholz   

                                        account_data  \
0  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
1  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
2  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
3  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
4  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   

         

In [3]:
# Reply Dataframe
# Importing Data into final Dataframe for ML
replyDataFrame = pd.DataFrame(columns = ["tweetid", "date", "time", "user", "text"])
print(replyDataFrame.head())
iterator = 0

for line in olafScholzJsonLines:
    keyResponse = line["response"]
    data = keyResponse["data"]
    newDataRow = [None, None, None, None, None]
    userName = line["account_name"]
    newDataRow[4] = userName
    keyResponse = line["response"]
    data = keyResponse["data"]


    for tweet in data:

        tweetTarget = 1
       
        tweetID = tweet["id"]
        authorID = tweet["author_id"]
        tweetDate = tweet["created_at"][0:10]
        tweetTime = tweet["created_at"][11:19]
        tweetText = tweet["text"]        
        newDataRow[0] = tweetID
        newDataRow[1] = tweetDate
        newDataRow[2] = tweetTime
        newDataRow[3] = authorID
        newDataRow[4] = tweetText
        replyDataFrame.loc[len(replyDataFrame)] = newDataRow
    iterator += 1


Empty DataFrame
Columns: [tweetid, date, time, user, text]
Index: []


In [4]:
#print(dir(replyDataFrame))
print(len(replyDataFrame))
print(replyDataFrame.shape)
print(replyDataFrame.head())

6419
(6419, 5)
               tweetid        date      time      user  \
0  1515588838562971650  2022-04-17  07:11:52  38150247   
1  1514644785818058757  2022-04-14  16:40:32  38150247   
2  1514291545742184451  2022-04-13  17:16:53  38150247   
3  1514256817613905929  2022-04-13  14:58:53  38150247   
4  1513955069867274245  2022-04-12  18:59:51  38150247   

                                                text  
0  RT @Bundeskanzler: Ich wünsche Ihnen und Ihren...  
1  RT @Bundeskanzler: Meine Solidarität gilt @Kar...  
2  RT @Bundeskanzler: Sie haben Hilfe organisiert...  
3  RT @Bundeskanzler: Verantwortung übernehmen he...  
4  RT @Bundeskanzler: Die Sanktionen gegen #Russl...  


In [5]:
print("----")
print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].sort_values("date",ascending = "true").head())

----
                tweetid        date      time      user  \
53  1493195813245313024  2022-02-14  12:09:58  38150247   
52  1493599012691365903  2022-02-15  14:52:09  38150247   
51  1493602211829587986  2022-02-15  15:04:51  38150247   
50  1494003008283287560  2022-02-16  17:37:29  38150247   
49  1494245849085493251  2022-02-17  09:42:26  38150247   

                                                 text  
53  RT @Bundeskanzler: Auf dem Weg in die #Ukraine...  
52  RT @Bundeskanzler: Der Frieden und die Sicherh...  
51  RT @Bundeskanzler: Ich habe Präsident #Putin i...  
50  RT @Bundeskanzler: Die #Corona-Regeln zeigen d...  
49  RT @Bundeskanzler: Gestern Abend habe ich mit ...  


In [6]:
#testing
print(replyDataFrame["text"].str.contains("RT @Bundeskanzler").head())
print(replyDataFrame["text"].str.contains("RT @Bundeskanzler").count())

0    True
1    True
2    True
3    True
4    True
Name: text, dtype: bool
6419


In [7]:
#cleansedTweetDF = replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:") or replyDataFrame["text"].str.contains("!RT @") ]
print("Size of dataframe :\n"+ str(replyDataFrame.count())+"\n----")
maskBundesKanz = replyDataFrame["text"].str.contains("RT @Bundeskanzler:")
maskNoRT = replyDataFrame["text"].str.contains("RT @")==False
print("Size of dataframe with RT @Bundeskanzler:\n" + str(replyDataFrame[maskBundesKanz].count())+"\n----")
print("Size of dataframe without RT @:\n" + str(replyDataFrame[maskNoRT].count())+"\n----")
#cleansedTweetDF contains all Tweets posted directly by Olaf before or after being Kanzler
cleansedTweetDF = replyDataFrame[maskBundesKanz | maskNoRT]
print("Size of cleansedTweetDF @:\n" + str(cleansedTweetDF.count())+"\n----")
print(cleansedTweetDF.head())
#cleansedTweetDF has "RT @Bundeskanzler:" still in the tweet. To not have any issues with ML we removed that part of the Tweet in the following parts

Size of dataframe :
tweetid    6419
date       6419
time       6419
user       6419
text       6419
dtype: int64
----
Size of dataframe with RT @Bundeskanzler:
tweetid    48
date       48
time       48
user       48
text       48
dtype: int64
----
Size of dataframe without RT @:
tweetid    4071
date       4071
time       4071
user       4071
text       4071
dtype: int64
----
Size of cleansedTweetDF @:
tweetid    4119
date       4119
time       4119
user       4119
text       4119
dtype: int64
----
               tweetid        date      time      user  \
0  1515588838562971650  2022-04-17  07:11:52  38150247   
1  1514644785818058757  2022-04-14  16:40:32  38150247   
2  1514291545742184451  2022-04-13  17:16:53  38150247   
3  1514256817613905929  2022-04-13  14:58:53  38150247   
4  1513955069867274245  2022-04-12  18:59:51  38150247   

                                                text  
0  RT @Bundeskanzler: Ich wünsche Ihnen und Ihren...  
1  RT @Bundeskanzler: Meine Solidaritä

In [8]:
# Removing the Text \"RT @Bundeskanzler:\" from the text\n
print("ReplyDataframe: \n" + str(replyDataFrame[maskBundesKanz].count()))
cleansedAtBundKanzDF = replyDataFrame[maskBundesKanz].copy()
cleansedAtBundKanzDF.text = cleansedAtBundKanzDF.text.str.strip("RT @Bundeskanzler: ")
print(cleansedAtBundKanzDF.head())                                                           

ReplyDataframe: 
tweetid    48
date       48
time       48
user       48
text       48
dtype: int64
               tweetid        date      time      user  \
0  1515588838562971650  2022-04-17  07:11:52  38150247   
1  1514644785818058757  2022-04-14  16:40:32  38150247   
2  1514291545742184451  2022-04-13  17:16:53  38150247   
3  1514256817613905929  2022-04-13  14:58:53  38150247   
4  1513955069867274245  2022-04-12  18:59:51  38150247   

                                                text  
0  Ich wünsche Ihnen und Ihren Familien frohe #Os...  
1  Meine Solidarität gilt @Karl_Lauterbach: Jeder...  
2  Sie haben Hilfe organisiert, Kräfte gebündelt ...  
3  Verantwortung übernehmen heißt: Deutschland li...  
4  Die Sanktionen gegen #Russland sind notwendig ...  


In [9]:
#appending the sliced versions to the Dataframe and resulting in the expected 4119 Tweets
cleansedTweetDFShort = replyDataFrame[maskNoRT].append(cleansedAtBundKanzDF)
print(cleansedTweetDFShort.count())
print(cleansedTweetDFShort.sort_values("date",ascending = False).head())

tweetid    4119
date       4119
time       4119
user       4119
text       4119
dtype: int64
               tweetid        date      time      user  \
0  1515588838562971650  2022-04-17  07:11:52  38150247   
1  1514644785818058757  2022-04-14  16:40:32  38150247   
2  1514291545742184451  2022-04-13  17:16:53  38150247   
3  1514256817613905929  2022-04-13  14:58:53  38150247   
4  1513955069867274245  2022-04-12  18:59:51  38150247   

                                                text  
0  Ich wünsche Ihnen und Ihren Familien frohe #Os...  
1  Meine Solidarität gilt @Karl_Lauterbach: Jeder...  
2  Sie haben Hilfe organisiert, Kräfte gebündelt ...  
3  Verantwortung übernehmen heißt: Deutschland li...  
4  Die Sanktionen gegen #Russland sind notwendig ...  


In [10]:
# Tobias 29.05.22 and 30.05.22

# Starting preparing for machine learning

# Making statement text in lower case

cleansedTweetDFShort

#Copy for emotion analysis

emotionDF = cleansedTweetDFShort.copy()

def clean(text):
    cleansedTweetDFShort['text'] = cleansedTweetDFShort['text'].str.replace('@[A-Za-z0-9]+\s?', '', regex=True)
    text = re.sub(r'@[A-Za-z0-9]+\s?', '', text) #Removed Mentions
    text = re.sub(r'#', '', text) #Removed #
    text = re.sub(r'(.)1+', r'1', text) #cleaned single letters
    text = re.sub('((www.[^s]+)|(https?://[^s]+))','',text) #Removes links
    text = re.sub('@','',text) #Remove @
    
    return text

cleansedTweetDFShort['text'] = cleansedTweetDFShort['text'].apply(clean)

cleansedTweetDFShort

,tweetid,date,time,user,text
12,1508125473842335749,2022-03-27,16:55:07,38150247,Die Saarländerinnen und Saarländer haben sich ...
21,1504398354565976070,2022-03-17,10:04:52,38150247,Meine persönliche Position ist längst bekannt:...
54,1492872048120668161,2022-02-13,14:43:27,38150247,"Ich freue mich, wenn Sie und Ihr mir ab heute ..."
55,1492074452796022791,2022-02-11,09:54:05,38150247,Für die aktuelle Welle der Pandemie gibt es An...
56,1492074451432873985,2022-02-11,09:54:05,38150247,Die Corona-Pandemie und auch die Flutkatastrop...
...,...,...,...,...,...
49,1494245849085493251,2022-02-17,09:42:26,38150247,Gestern Abend habe ich mit Biden zur Situation...
50,1494003008283287560,2022-02-16,17:37:29,38150247,Die Corona-Regeln zeigen die erhoffte Wirkung....
51,1493602211829587986,2022-02-15,15:04:51,38150247,Ich habe Präsident Putin in Moskau verdeutlich...
52,1493599012691365903,2022-02-15,14:52:09,38150247,Der Frieden und die Sicherheit in Europa sind ...


In [11]:
cleansedTweetDFShort = cleansedTweetDFShort.astype({'text':'string'})
print(cleansedTweetDFShort.dtypes)
cleansedTweetDFShort.head()

tweetid    object
date       object
time       object
user       object
text       string
dtype: object


,tweetid,date,time,user,text
12,1508125473842335749,2022-03-27,16:55:07,38150247,Die Saarländerinnen und Saarländer haben sich ...
21,1504398354565976070,2022-03-17,10:04:52,38150247,Meine persönliche Position ist längst bekannt:...
54,1492872048120668161,2022-02-13,14:43:27,38150247,"Ich freue mich, wenn Sie und Ihr mir ab heute ..."
55,1492074452796022791,2022-02-11,09:54:05,38150247,Für die aktuelle Welle der Pandemie gibt es An...
56,1492074451432873985,2022-02-11,09:54:05,38150247,Die Corona-Pandemie und auch die Flutkatastrop...


In [12]:
# Philip 04.06.2022 Language Detection

languageDF = cleansedTweetDFShort.copy()
languageDF.reset_index(inplace=True, drop=True)

In [13]:
#trying different text detection approaches
#https://stackoverflow.com/questions/39142778/python-how-to-determine-the-language
# -> https://pypi.org/project/gcld3/ -> requires https://grpc.io/docs/protoc-installation/ -> Windows Install https://www.geeksforgeeks.org/how-to-install-protocol-buffers-on-windows/

In [14]:
# TextBlob Approach - Doesnt work due to Error 400 - Seems to be an issue with the Google API
# https://www.geeksforgeeks.org/detect-an-unknown-language-using-python/
"""from textblob import TextBlob
for tweet in languageDF.loc[0:1,"text"]:
    print(tweet)
    print(type(TextBlob(tweet)))
    # To get detect_language() https://stackoverflow.com/questions/69207838/textblob-detect-language-function-not-working
    print("Language: " + str(TextBlob(tweet).detect_language()))
    print("----")"""

'from textblob import TextBlob\nfor tweet in languageDF.loc[0:1,"text"]:\n    print(tweet)\n    print(type(TextBlob(tweet)))\n    # To get detect_language() https://stackoverflow.com/questions/69207838/textblob-detect-language-function-not-working\n    print("Language: " + str(TextBlob(tweet).detect_language()))\n    print("----")'

In [15]:
#LangDetect
from langdetect import detect
languageDF["Language"] = detect(str(languageDF.text))
print(languageDF.head())

               tweetid        date      time      user  \
0  1508125473842335749  2022-03-27  16:55:07  38150247   
1  1504398354565976070  2022-03-17  10:04:52  38150247   
2  1492872048120668161  2022-02-13  14:43:27  38150247   
3  1492074452796022791  2022-02-11  09:54:05  38150247   
4  1492074451432873985  2022-02-11  09:54:05  38150247   

                                                text Language  
0  Die Saarländerinnen und Saarländer haben sich ...       de  
1  Meine persönliche Position ist längst bekannt:...       de  
2  Ich freue mich, wenn Sie und Ihr mir ab heute ...       de  
3  Für die aktuelle Welle der Pandemie gibt es An...       de  
4  Die Corona-Pandemie und auch die Flutkatastrop...       de  


In [16]:
print("Amount of different Languages found: " + str(languageDF["Language"].nunique()))
uniqueLang = languageDF["Language"].unique()
print(uniqueLang)
# End of Language Detection

Amount of different Languages found: 1
['de']


In [ ]:
# Extra DF for wordcloud

cleansedTweetDFShort10 = cleansedTweetDFShort[:100].copy()
cleansedTweetDFShort10

In [ ]:
#Get Polarity and Subjectivity

# Subjectivity quantifies the amount of personal opinion and factual information contained in the text. 
# The higher subjectivity means that the text contains personal opinion rather than factual information. 
# 0 means low personal opinion and 1 a lot of personal opinion

def getS (text):
    return TextBlobDE(text).sentiment.subjectivity

# Polarity lies between [-1,1], -1 defines a negative sentiment and 1 defines a positive sentiment.

def getP (text):
    return TextBlobDE(text).sentiment.polarity


cleansedTweetDFShort10['Personal Opinion (Subjectivity)'] = cleansedTweetDFShort10['text'].apply(getS)
cleansedTweetDFShort10['Sentiment (Polarity)'] = cleansedTweetDFShort10['text'].apply(getP)

cleansedTweetDFShort10

In [ ]:
# Plot Word Cloud not cleaned

plt.figure(figsize = (20,20))

allWords = ' '.join([txt for txt in cleansedTweetDFShort10['text']])
wordCloud = WordCloud(max_words=500, width=2000, height = 1000, 
                      max_font_size = 1000, collocations=False).generate(allWords)

plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Defining set containing all stopwords in German. (Stopwords: Word with no relevance for analysis)
# Source for Stopwordlist: https://github.com/stopwords-iso/stopwords-de/blob/master/stopwords-de.json
# Cleaning text from stopwords

# Create copy to work with
cleansedTweetDFShort10copy = cleansedTweetDFShort10.copy()

# Making statement text in lower case
cleansedTweetDFShort10copy['text']=cleansedTweetDFShort10copy['text'].str.lower()

with open('stopwords-de.json','r') as file:
    stopwordlistgerman = json.load(file)

STOPWORDS = set(stopwordlistgerman)

def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

cleansedTweetDFShort10copy['text'] = cleansedTweetDFShort10copy['text'].apply(lambda text: cleaning_stopwords(text))
cleansedTweetDFShort10copy['text'].head(10)

In [ ]:
# Cleaning text from english stopwords
# Source for Stopwordlist: https://countwordsfree.com/stopwords
# Cleaning text from stopwords
with open('stop_words_english.json','r') as file:
    stopwordlistenglish = json.load(file)

STOPWORDS = set(stopwordlistenglish)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
cleansedTweetDFShort10copy['text'] = cleansedTweetDFShort10copy['text'].apply(lambda text: cleaning_stopwords(text))
cleansedTweetDFShort10copy['text'].head(10)

In [ ]:
#Remove punctations

import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
cleansedTweetDFShort10copy['text']= cleansedTweetDFShort10copy['text'].apply(lambda x: cleaning_punctuations(x))
cleansedTweetDFShort10copy['text'].head(10)

In [ ]:
# Applying Stemming

import nltk
from nltk.stem import PorterStemmer
 
porter = PorterStemmer()

st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
cleansedTweetDFShort10copy['text']= cleansedTweetDFShort10copy['text'].apply(lambda x: stemming_on_text(x))
cleansedTweetDFShort10copy = cleansedTweetDFShort10copy.astype({'text':'string'})
cleansedTweetDFShort10copy['text'].head()

In [ ]:
# Plot Word Cloud after cleaning
plt.figure(figsize = (20,20))
cleanedwords = ' '.join([txt for txt in cleansedTweetDFShort10copy['text']])
wordCloud = WordCloud(max_words = 500, width=2000, height = 1000, max_font_size = 1000, 
                      collocations=False).generate(cleanedwords)

plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Sentiment Analysis with TextBlobDE

# Create a function to compute analysis

def getA(score):
    if score < 0:
        return 'negative'
    elif score == 0:
        return 'neutral'
    else:
        return 'positive'

cleansedTweetDFShort10['Analysis'] = cleansedTweetDFShort10['Sentiment (Polarity)'].apply(getA)
cleansedTweetDFShort10

In [ ]:
sortedDf = cleansedTweetDFShort10.sort_values(by=['Sentiment (Polarity)'])

#sortedDf = sortedDf.astype({'Analysis':'string'})
print(sortedDf.dtypes)
sortedDf.reset_index(inplace=True, drop=True) # Reset index
sortedDf.head()

In [ ]:
# Show texts with Polarity and Analysis 

# Problem erkannt: Er schreibt sowohl englisch als auch deutsch

sortedDf

for i in range (0, sortedDf.shape[0]):
    print(sortedDf['text'][i])
    print(sortedDf['Personal Opinion (Subjectivity)'][i])
    print(sortedDf['Sentiment (Polarity)'][i])
    print(sortedDf['Analysis'][i])
    print()

In [ ]:
# Print Positive tweets

j=1;

for i in range(0, sortedDf.shape[0]):
    if(sortedDf['Analysis'][i] == 'positive'):
        print(str(j) + ') ' + sortedDf['text'][i])
        print(sortedDf['Personal Opinion (Subjectivity)'][i])
        print(sortedDf['Sentiment (Polarity)'][i])
        print(sortedDf['Analysis'][i])
        print()
        j=j+1

In [ ]:
# Print Negative tweets

j=1;

for i in range(0, sortedDf.shape[0]):
    if(sortedDf['Analysis'][i] == 'negative'):
        print(str(j) + ') ' + sortedDf['text'][i])
        print(sortedDf['Personal Opinion (Subjectivity)'][i])
        print(sortedDf['Sentiment (Polarity)'][i])
        print(sortedDf['Analysis'][i])
        print()
        j=j+1

In [ ]:
#Plot polarity and subjectivity

plt.figure(figsize=(8,6))

for i in range (0, sortedDf.shape[0]):
    plt.scatter(sortedDf['Sentiment (Polarity)'][i], sortedDf['Personal Opinion (Subjectivity)'][i], color='Green')
                
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
# Get the percentage of positive tweets

ptweets = sortedDf[sortedDf.Analysis =="positive"] 

print("Positive Tweets in percent: "+str(round((ptweets.shape[0]/sortedDf.shape[0])*100, 1)) + '%')

In [ ]:
# Get the negative of positive tweets

neutweets = sortedDf[sortedDf.Analysis =="neutral"] 

print("Neutral Tweets in percent: "+str(round((neutweets.shape[0]/sortedDf.shape[0])*100, 1)) + '%')

In [ ]:
# Get the negative of positive tweets

ntweets = sortedDf[sortedDf.Analysis =="negative"] 

print("Negative Tweets in percent: "+str(round((ntweets.shape[0]/sortedDf.shape[0])*100, 1)) + '%')

In [ ]:
# Show Analysis

sortedDf['Analysis'].value_counts()

plt.title('Sentiment Analysis in %')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
sortedDf['Analysis'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
# Tobias 03.06.2022
# Sentiment and Emotion Analysis

warnings.filterwarnings("ignore", category=DeprecationWarning)

emotionDF

In [ ]:
# Text processing

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag

STOPLIST = set(stopwords.words('german'))
SYMBOLS = " ".join(string.punctuation).split(" ") + \
["-", "...", "”", "``", ",", ".", ":", "''","#","@"]

# The NLTK lemmatizer and stemmer classes
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('german')



In [ ]:
# I use the POS tagging from NLTK to retain only adjectives, verbs, adverbs 
# and nouns as a base for for lemmatization.
def get_lemmas(tweet): 
    
    # A dictionary to help convert Treebank tags to WordNet
    treebank2wordnet = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'}
    
    postag = ''
    lemmas_list = []
    
    for word, tag in pos_tag(word_tokenize(tweet)):
        if tag.startswith("JJ")     \
            or tag.startswith("RB") \
            or tag.startswith("VB") \
            or tag.startswith("NN"):
                
            try:
                postag = treebank2wordnet[tag[:2]]
            except:
                postag = 'n'                
                            
            lemmas_list.append(lemmatizer.lemmatize(word.lower(), postag))    
    
    return lemmas_list

In [ ]:
# This is the function to clean and filter the tokens in each tweet
def clean_tweet(tokens):
    
    filtered = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            if token not in STOPLIST:
                if token[0] not in SYMBOLS:
                    if not token.startswith('http'):
                        if  '/' not in token:
                            if  '-' not in token:
                                filtered.append(token)
                                        
    return filtered

In [ ]:
# Starts the lemmatization process
def get_lemmatized(tweet):
   
    all_tokens_string = ''
    filtered = []
    tokens = []

    # lemmatize
    tokens = [token for token in get_lemmas(tweet)]
    
    # filter
    filtered = clean_tweet(tokens)

    # join everything into a single string
    all_tokens_string = ' '.join(filtered)
    
    return all_tokens_string

In [ ]:
# get the lemmatized tweets and puts the result in an "edited" text column
# for future use in this script
edited = ''
for i, row in emotionDF.iterrows():
    edited = get_lemmatized(emotionDF.loc[i]['text'])
    if len(edited) > 0:
        emotionDF.at[i,'edited'] = edited
    else:
        emotionDF.at[i,'edited'] = None     

In [ ]:
# After lemmatization, some tweets may end up with the same words
# Let's make sure that we have no duplicates
emotionDF.drop_duplicates(subset=['edited'], inplace=True)
emotionDF.dropna(inplace=True)

In [ ]:
# Using apply/lambda to create a new column with the number of words in each tweet
emotionDF['word_count'] = emotionDF.apply(lambda x: len(x['text'].split()),axis=1)
t = pd.DataFrame(emotionDF['word_count'].describe()).T
t

In [ ]:
emotionDF

In [ ]:
# Sentiment Analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

emotionDF['date']=pd.to_datetime(emotionDF['date']) 
emotionDF.sort_values('date', inplace=True, ascending=True)
emotionDF = emotionDF.reset_index(drop=True)

In [ ]:
# To avoid repetitions in our code, here are some plotting functions 
# that will be called often ...

def plot_sentiment_period(df, info):
    
    # Using the mean values of sentiment for each period
    df1 = df.groupby(df['datetime'].dt.to_period(info['period'])).mean()

    df1.reset_index(inplace=True)
    df1['datetime'] = pd.PeriodIndex(df1['datetime']).to_timestamp()
    plot_df = pd.DataFrame(df1, df1.index, info['cols'])

    plt.figure(figsize=(15, 10))
    ax = sns.lineplot(data=plot_df, linewidth = 3, dashes = False)
    plt.legend(loc='best', fontsize=15)
    plt.title(info['title'], fontsize=20)
    plt.xlabel(info['xlab'], fontsize=15)
    plt.ylabel(info['ylab'], fontsize=15)
    plt.tight_layout()
    plt.savefig('images/' + info['fname'])    
    return


def plot_fractions(props, title, fname):
    
    plt1 = props.plot(kind='bar', stacked=False, figsize=(16,5), colormap='Spectral') 
    plt.legend(bbox_to_anchor=(1.005, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Online storm', fontweight='bold', fontsize=18)
    plt.xticks(rotation=0,fontsize=14)
    #plt.ylim(0, 0.5)
    plt.ylabel('Fraction of Tweets', fontweight='bold', fontsize=18)
    plt1.set_title(label=title, fontweight='bold', size=20)
    plt.tight_layout()
    plt.savefig('images/' + fname + '.png')
    
    return


def plot_frequency_chart(info):
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.set_context("notebook", font_scale=1)    
    ax = sns.barplot(x=info['x'], y=info['y'], data=info['data'], palette=(info['pal']))
    ax.set_title(label=info['title'], fontweight='bold', size=18)
    plt.ylabel(info['ylab'], fontsize=16)
    plt.xlabel(info['xlab'], fontsize=16)
    plt.xticks(rotation=info['angle'],fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.savefig('images/' + info['fname'])
    
    return

In [ ]:
# Calling VADER

analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Get VADER Compound value for sentiment intensity
emotionDF['sentiment_intensity'] = [analyzer.polarity_scores(v)['compound'] for v in emotionDF['edited']]

In [ ]:
# This function returns the sentiment category
def get_sentiment(intensity):
    if intensity >= 0.05:
        return 'Positive'
    elif (intensity >= -0.05) and (intensity < 0.05):
        return 'Neutral'
    else:
        return 'Negative'

# Using pandas apply/lambda to speed up the process
emotionDF['sentiment'] = emotionDF.apply(lambda x: get_sentiment(x['sentiment_intensity']),axis=1)
emotionDF

In [ ]:
#Translate word using Google translator for Emotionanalysis

import googletrans
from googletrans import Translator

translater = Translator() #German: 'de'

#print(googletrans.LANGUAGES) 

#Test
translator = Translator()
translation = translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en')
print(translation.text)

In [ ]:
emotiontest = emotionDF[:3].copy()
emotiontest['edited'] = emotiontest['edited'].astype(str)
emotiontest['translated'] = emotiontest['edited'].apply(translater.translate, src ="de", dest='en').apply(getattr, args=('text',))

# Problem: Number of translations is limited

In [ ]:
# Emotionanalysis

import termcolor
import sys
from termcolor import colored, cprint
plt.style.use('fivethirtyeight')
# Importing the data from the NCR lexicon
ncr = pd.read_csv('NCR-lexicon.csv', sep =';')

In [ ]:
# Let's create a list of the emotions
emotions = ['Anger', 'Anticipation','Disgust','Fear', 'Joy','Sadness', 'Surprise', 'Trust']
# Join all the edited tweets in one single string
joined_string = emotiontest(emotiontest['edited'])

# Get tokens
tokens = joined_string.split(' ')